<a href="https://colab.research.google.com/github/yms06034/ai-tech-interview/blob/main/csv_to_parquet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas
!pip install pyarrow

In [ ]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

In [ ]:
csv_file = './email.csv'
parquet_file = "./email.parquet"
chunksize = 500
csv_stream = pd.read_csv(csv_file, sep=';', chunksize=chunksize, low_memory=False)
chunk=next(iter(csv_stream))

In [ ]:
test_csv= pd.read_csv('./email.csv', sep=';')
test_csv.columns

In [ ]:
parquet_schema_old = pa.Table.from_pandas(df=chunk).schema
parquet_schema_new = pa.schema([
    ('Login email', pa.string()),
    ('Identifier', pa.string()),
    ('First name',pa.string()),
    ('Last name', pa.string())
])
parquet_schema_old == parquet_schema_new

In [ ]:
csv_file = './email.csv'
parquet_file = "./email.parquet"
chunksize = 100_000

csv_stream = pd.read_csv(csv_file, sep=';', chunksize=chunksize, low_memory=False)

for i, chunk in enumerate(csv_stream):
    print("Chunk", i)
    if i == 0:
        parquet_schema = pa.Table.from_pandas(df=chunk).schema
        parquet_writer = pq.ParquetWriter(parquet_file, parquet_schema, compression='snappy')
    table = pa.Table.from_pandas(chunk, schema=parquet_schema)
    parquet_writer.write_table(table)

parquet_writer.close()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, LongType

spark = SparkSession.builder.appName("CSV to Parquet").getOrCreate()

schema = StructType([
    StructField("timestamp", StringType(), True),
    StructField("site", StringType(), True),
    StructField("requests", LongType(), True)
])

df = spark.read \
    .schema(schema) \
    .option("header", "true") \
    .option("delimiter", "\t") \
    .csv("/user/hduser/wikipedia/pageviews-by-second-tsv")

df.write.parquet("/user/hduser/wikipedia/pageviews-by-second-parquet")

spark.stop()